In [ ]:
import torch
from collections import defaultdict
import numpy as np
import mcubes
import trimesh

from models.rendering import *
from models.nerf import *

from datasets import dataset_dict

from utils import load_ckpt
import tqdm

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="8"

# Load model and data

In [ ]:
img_wh = (4032, 3024) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = 'firekeeper_hr' # whatever you want
root_dir = '../pretrained/firekeeper/' # the folder containing data
ckpt_path = 'ckpts/firekeeper_hr_fastnerf/epoch=12.ckpt' # the model path
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*32
dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval()

os.makedirs('output', exist_ok=True)

# Search for tight bounds of the object (trial and error!)

In [ ]:
############################################################################################
## calculate uvws
############################################################################################

N = 768 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
        
## Attention! the ranges MUST have the same length!
xmin, xmax = -1.2, 1.2 # left/right range
ymin, ymax = -1.2, 1.2 # forward/backward range
zmin, zmax = -1.2, 1.2 # up/down range

# sigma and sigma
x = np.linspace(xmin, xmax, N, endpoint=False)
y = np.linspace(ymin, ymax, N, endpoint=False)
z = np.linspace(zmin, zmax, N, endpoint=False)
xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3))
dir_ = torch.zeros_like(xyz_)

with torch.no_grad():
    B = xyz_.shape[0]
    uvw, sigma = [], []
    for i in tqdm.trange(0, B, chunk):
        xyz_embedded = embedding_xyz(xyz_[i:i+chunk].cuda()) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:i+chunk].cuda()) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        uvw_, beta_, sigma_ = nerf_fine(xyzdir_embedded, return_components=True)
        uvw.append(uvw_.cpu()) 
        sigma.append(sigma_.cpu()) 
    uvw = torch.cat(uvw, 0).numpy().astype(np.float32).reshape(N, N, N, -1) # [NNN, 3, 8] --> [N, N, N, 3*8]
    sigma = torch.cat(sigma, 0).numpy().astype(np.float32).reshape(N, N, N, -1) # [NNN, 1] --> [N, N, N, 1]

sigma = np.maximum(sigma, 0)

print(uvw.shape, uvw.dtype)
print(sigma.shape, sigma.dtype)



In [ ]:
sigma_thresh = 0
mask = (sigma[:, :, :, 0] > sigma_thresh) # [N, N, N], 0/1

# save link list like sparse matrix
coords = np.nonzero(mask)
nnz = coords[0].shape[0]
print(nnz, nnz / np.prod(mask.shape))

inds = -np.ones_like(mask, dtype=np.int32)
inds[coords] = np.arange(nnz)

uvws = np.concatenate([
    uvw[coords], # [nnz, 24]
    sigma[coords],
], axis=1).astype(np.float32)

print(inds.shape, uvws.shape)
np.save(f'output/{scene_name}_inds_{N}_{sigma_thresh}.npy', inds)
np.save(f'output/{scene_name}_uvws_{N}_{sigma_thresh}.npy', uvws)
print('saved')

In [ ]:
############################################################################################
## calculate beta (spherical ver.) (anyway, seems buggy)
############################################################################################

M = 512 # for angles

theta = np.linspace(0, np.pi, M, endpoint=True) # [0, 180], elevation
phi = np.linspace(-np.pi, np.pi, M, endpoint=False) # [-180, 180], azimuth
dir_spherical = np.stack(np.meshgrid(theta, phi), -1).reshape(-1, 2) # [M * 2M, 2]
dir_ = torch.FloatTensor(np.stack([
    np.cos(dir_spherical[:, 1]) * np.sin(dir_spherical[:, 0]), 
    np.sin(dir_spherical[:, 1]) * np.sin(dir_spherical[:, 0]), 
    np.cos(dir_spherical[:, 0]),
], axis=1)) # [M*2M, 3] normalized 
xyz_ = torch.zeros_like(dir_)

with torch.no_grad():
    B = dir_.shape[0]
    beta = []
    for i in tqdm.trange(0, B, chunk):
        end = min(B, i+chunk)
        xyz_embedded = embedding_xyz(xyz_[i:end].cuda()) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:end].cuda()) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        uvw_, beta_, sigma_ = nerf_fine(xyzdir_embedded, return_components=True)
        beta.append(beta_.cpu()) 
        
    beta = torch.cat(beta, 0).numpy().astype(np.float32).reshape(M, M, -1) # [M, M, 8]

print(beta.shape, beta.dtype)

In [ ]:
np.save(f'output/{scene_name}_beta_{M}_spherical.npy', beta)
print(f'saved {beta.shape}')

In [ ]:
############################################################################################
## calculate beta (cartesian ver.)
############################################################################################

M = 200 # for angles

# beta
nx = np.linspace(-1, 1, M, endpoint=False)
ny = np.linspace(-1, 1, M, endpoint=False)
nz = np.linspace(-1, 1, M, endpoint=False)
dir_ = np.stack(np.meshgrid(nx, ny, nz), -1).reshape(-1, 3)
dir_ = dir_ / (np.linalg.norm(dir_, ord=2, axis=-1, keepdims=True) + 1e-6)
dir_ = torch.FloatTensor(dir_).cuda()
xyz_ = torch.zeros_like(dir_)

#chunk = 1
chunk = 1024*32

with torch.no_grad():
    B = dir_.shape[0]
    beta = []
    for i in tqdm.trange(0, B, chunk):
        end = min(B, i+chunk)
        xyz_embedded = embedding_xyz(xyz_[i:end].cuda()) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:end].cuda()) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        uvw_, beta_, sigma_ = nerf_fine(xyzdir_embedded, return_components=True)
        beta.append(beta_.cpu()) 
        
    beta = torch.cat(beta, 0).numpy().astype(np.float32).reshape(M, M, M, -1) # [M, M, M, 8]

print(beta.shape, beta.dtype)

In [ ]:
np.save(f'output/{scene_name}_beta_{M}_cart.npy', beta)
print(f'saved {beta.shape}')

In [ ]:
###########################################################################################
## visualize mesh (maybe slow)
###########################################################################################
sigma_thresh = 20
vertices, triangles = mcubes.marching_cubes(sigma.copy()[:, :, :, 0], sigma_thresh)
mesh = trimesh.Trimesh(vertices/N, triangles)
mesh.show()